In [1]:
""" includes a series of tests, to get a good sense of time usage, memory usage and p-value (statistical consistency): """

import random
import secrets
import time
import tracemalloc
from scipy.stats import kstest
import numpy as np

# Parameters
num_samples = 100000

# Test random.random()
def test_random_random():
    start_time = time.time()
    numbers = [random.random() for _ in range(num_samples)]
    elapsed_time = time.time() - start_time
    return numbers, elapsed_time

# Test LCG
class LCG:
    def __init__(self, seed=1, a=1103515245, c=12345, m=2**31):
        self.state = seed
        self.a = a
        self.c = c
        self.m = m

    def random(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m

def test_lcg():
    lcg = LCG()
    start_time = time.time()
    numbers = [lcg.random() for _ in range(num_samples)]
    elapsed_time = time.time() - start_time
    return numbers, elapsed_time

# Test CSPRNG
def test_csprng():
    start_time = time.time()
    numbers = [secrets.SystemRandom().random() for _ in range(num_samples)]
    elapsed_time = time.time() - start_time
    return numbers, elapsed_time

# Halton sequence implementation
class Halton:
    def __init__(self, base):
        self.base = base
        self.index = 0

    def next(self):
        self.index += 1
        result = 0
        f = 1.0 / self.base
        i = self.index
        while i > 0:
            result += f * (i % self.base)
            i //= self.base
            f /= self.base
        return result

def test_halton():
    halton = Halton(2)  # Using base 2 for simplicity
    start_time = time.time()
    numbers = [halton.next() for _ in range(num_samples)]
    elapsed_time = time.time() - start_time
    return numbers, elapsed_time

# Memory usage measurement
def measure_memory_usage(generator_func):
    tracemalloc.start()
    numbers, elapsed_time = generator_func()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak - current
    return numbers, elapsed_time, memory_usage

# Consistency Test (using Kolmogorov-Smirnov test for uniform distribution)
def consistency_test(numbers):
    return kstest(numbers, 'uniform').pvalue

# Run tests
random_numbers, random_time, random_memory = measure_memory_usage(test_random_random)
lcg_numbers, lcg_time, lcg_memory = measure_memory_usage(test_lcg)
csprng_numbers, csprng_time, csprng_memory = measure_memory_usage(test_csprng)
halton_numbers, halton_time, halton_memory = measure_memory_usage(test_halton)

# Consistency tests
random_pvalue = consistency_test(random_numbers)
lcg_pvalue = consistency_test(lcg_numbers)
csprng_pvalue = consistency_test(csprng_numbers)
halton_pvalue = consistency_test(halton_numbers)

# Print results
print("Random.random() - Time: {:.4f}s, Memory: {} bytes, P-value: {:.4f}".format(random_time, random_memory, random_pvalue))
print("LCG - Time: {:.4f}s, Memory: {} bytes, P-value: {:.4f}".format(lcg_time, lcg_memory, lcg_pvalue))
print("CSPRNG - Time: {:.4f}s, Memory: {} bytes, P-value: {:.4f}".format(csprng_time, csprng_memory, csprng_pvalue))
print("Halton Sequence - Time: {:.4f}s, Memory: {} bytes, P-value: {:.4f}".format(halton_time, halton_memory, halton_pvalue))


Random.random() - Time: 0.0988s, Memory: 176 bytes, P-value: 0.3865
LCG - Time: 0.3873s, Memory: 668 bytes, P-value: 0.0868
CSPRNG - Time: 0.5103s, Memory: 3120 bytes, P-value: 0.7292
Halton Sequence - Time: 2.1369s, Memory: 672 bytes, P-value: 1.0000
